In [ ]:
from tensorflow.keras.layers import Input, SimpleRNN, LSTM, GRU, Conv1D, Embedding, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import pdist, squareform
from tensorflow.keras import Model
import matplotlib.pyplot as plt
import tensorflow as tf
import random as rand
import numpy as np
import pronouncing
import markovify
import textstat
import math

In [ ]:
lyric_path = '/Users/patricknaylor/Desktop/Metal/Data/lyrics_1.txt'
with open(lyric_path, 'r') as file:
    song = (file.read())
    lyrics = song.replace('\ufeff', '').split("\n")



In [ ]:
markov_model = markovify.NewlineText(str("\n".join(lyrics)), well_formed=False, state_size=3)
sentence = markov_model.make_sentence(tries=100)
print(sentence)

In [ ]:
sequences = lyrics
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(sequences)

V = len(tokenizer.word_index)+1
seq = pad_sequences(tokenizer.texts_to_sequences(sequences), maxlen=30)
tokenizer.word_index.items()

In [ ]:
train_X, train_y = seq[:, :-1], tf.keras.utils.to_categorical(seq[:, -1], num_classes=V)

print(train_X.shape, train_y.shape)

In [ ]:
D = 512

#Simple RNN
T = train_X.shape[1]
i = Input(shape=(T,))
x = Embedding(V, D)(i)
x = Dropout(0.2)(x)
x = SimpleRNN(150)(x)
x = Dense(V, activation="softmax")(x)
rnn_model = Model(i, x)

adam = tf.keras.optimizers.Adam(learning_rate=0.001)

rnn_model.compile(optimizer=adam, metrics=["accuracy"], loss="categorical_crossentropy")
rnn_model.summary()

In [ ]:
from keras.callbacks import ReduceLROnPlateau , EarlyStopping
from tensorflow.keras.optimizers import Adam 


import warnings
warnings.filterwarnings('ignore')
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

es = EarlyStopping(monitor="loss", mode="min", verbose=1, patience=20)

In [8]:
rnn_r = rnn_model.fit(train_X, train_y, epochs=100,callbacks=[learning_rate_reduction,es])

1795/1796 [============================>.] - ETA: 0s - loss: 2.9021 - accuracy: 0.4911WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 73s 41ms/step - loss: 2.9021 - accuracy: 0.4911 - lr: 0.0010
Epoch 7/100
1795/1796 [============================>.] - ETA: 0s - loss: 2.3511 - accuracy: 0.5783WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 72s 40ms/step - loss: 2.3513 - accuracy: 0.5782 - lr: 0.0010
Epoch 8/100
1796/1796 [==============================] - ETA: 0s - loss: 1.9581 - accuracy: 0.6412WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 70s 39ms/step - loss: 1.9581 - accuracy: 0.6412 - lr: 0.0010
Epoch 9/100
1796/1796 [==============================] - ETA: 0s - loss: 1.6892 - accuracy: 0.6827WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 70s 39ms/step - loss: 1.6892 - accuracy: 0.6827 - lr: 0.0010
Epoch 10/100
1795/1796 [============================>.] - ETA: 0s - loss: 1.4935 - accuracy: 0.7171WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


...

1796/1796 [==============================] - 72s 40ms/step - loss: 0.8354 - accuracy: 0.8146 - lr: 0.0010
Epoch 74/100
1796/1796 [==============================] - ETA: 0s - loss: 0.8575 - accuracy: 0.8083WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 71s 39ms/step - loss: 0.8575 - accuracy: 0.8083 - lr: 0.0010
Epoch 75/100
1796/1796 [==============================] - ETA: 0s - loss: 0.8488 - accuracy: 0.8120WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 72s 40ms/step - loss: 0.8488 - accuracy: 0.8120 - lr: 0.0010
Epoch 76/100
1795/1796 [============================>.] - ETA: 0s - loss: 0.8522 - accuracy: 0.8105WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 1032s 575ms/step - loss: 0.8521 - accuracy: 0.8105 - lr: 0.0010
Epoch 77/100
1796/1796 [==============================] - ETA: 0s - loss: 0.8495 - accuracy: 0.8123WARNING:tensorflow:Learning rate reduction is conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy,lr


1796/1796 [==============================] - 72s 40ms/step - loss: 0.8495 - accuracy: 0.8123 - lr: 0.0010
Epoch 77: early stopping


In [9]:
def calc_readability(input_bars):
  avg_readability = 0
  for bar in input_bars:
    avg_readability += textstat.automated_readability_index(bar)
  return avg_readability / len(input_bars)

In [10]:
def calc_rhyme_density(bars):
  total_syllables = 0
  rhymed_syllables = 0
  for bar in bars:
    for word in bar.split():
      p = pronouncing.phones_for_word(word)
      if len(p) == 0:
        break
      syllables = pronouncing.syllable_count(p[0])
      total_syllables += syllables
      has_rhyme = False
      for rhyme in pronouncing.rhymes(word):
        if has_rhyme:
          break
        for idx, r_bar in enumerate(bars):
          if idx > 4:
            break
          if rhyme in r_bar:
            rhymed_syllables += syllables
            has_rhyme = True
            break
  return rhymed_syllables/total_syllables

In [11]:
def generate_bar(seed_phrase, model, length_of_bar):
  for i in range(length_of_bar):
    seed_tokens = pad_sequences(tokenizer.texts_to_sequences([seed_phrase]), maxlen=29)
    output_p = model.predict(seed_tokens)
    output_word = np.argmax(output_p, axis=1)[0]-1
    seed_phrase += " " + str(list(tokenizer.word_index.items())[output_word][0])
  return seed_phrase

In [12]:
def score_bar(input_bar, artists_bars, artists_avg_readability, artists_avg_rhyme_idx):
  gen_readability = textstat.automated_readability_index(input_bar)
  gen_rhyme_idx = calc_rhyme_density(input_bar)
  comp_bars = compare_bars(input_bar, artists_bars)

  # Scores based off readability, rhyme index, and originality. The lower the score the better.
  bar_score = (artists_avg_readability - gen_readability) + (artists_avg_rhyme_idx - gen_rhyme_idx) + comp_bars
  return bar_score

In [13]:
def compare_bars(input_bar, artists_bars):
  '''
    input_bars are the fire bars our AI generates
    artists_bars are the original bars for the artist

    The lower the score the better! We want unique bars
  '''
  # Converts sentences to matrix of token counts
  avg_dist = 0
  total_counted = 0
  for bar in artists_bars:
    v = CountVectorizer()
    # Vectorize the sentences
    word_vector = v.fit_transform([input_bar, bar])
    # Compute the cosine distance between the sentence vectors
    cos_dist = 1-pdist(word_vector.toarray(), 'cosine')[0]
    if not math.isnan(cos_dist):
      avg_dist += 1-pdist(word_vector.toarray(), 'cosine')[0]
      total_counted += 1
  return avg_dist/total_counted

In [20]:
def generate_song( model, artists_bars, length_of_bar, length_of_song=20, min_score_threshold=-0.2, max_score_threshold=0.2, tries=5):
  artists_avg_readability = calc_readability(artists_bars)
  artists_avg_rhyme_idx = calc_rhyme_density(artists_bars)
  fire_song = []
  cur_tries = 0
  candidate_bars = []

  while len(fire_song) < length_of_song:
    try:
        seed_sentence = markov_model.make_sentence(tries=100).split(" ")
        print('Seed Sentence: ', seed_sentence)
        seed_sentence = " ".join(seed_sentence[:5])
    except:
        pass
    cur_tries += 1
    bar = generate_bar(seed_sentence, model, rand.randrange(4, length_of_bar))
    #print(bar)
    bar_score = score_bar(bar, lyrics, artists_avg_readability, artists_avg_rhyme_idx) 
    candidate_bars.append((bar_score, bar))


    if bar_score <= max_score_threshold and bar_score >= min_score_threshold:
      fire_song.append(bar)
      cur_tries = 0
      print("Generated Bar: ", len(fire_song))

    if cur_tries >= tries:
      lowest_score = np.Infinity
      best_bar = ""
      for bar in candidate_bars:
        if bar[0] < lowest_score:
          best_bar = bar[1]
          candidate_bars = []
      
      fire_song.append(best_bar)
      print("Generated Bar: ", len(fire_song))
      cur_tries = 0
      
  print("Generated song with avg rhyme density: ", calc_rhyme_density(fire_song), "and avg readability of: ", calc_readability(fire_song))
  return fire_song

In [21]:
print('test')
rnn = generate_song( rnn_model, lyrics, length_of_bar =6 , tries=5)

print("Song Generated with SimpleRNN:")
for line in rnn:
  print(line)
print()

test
Seed Sentence:  ['We', 'all', 'want', 'to', 'be', 'punished', 'for', 'my', 'sins', 'at', 'hand']
1/1 [==============================] - 0s 12ms/step
Seed Sentence:  ['We', 'have', 'finally', 'found', 'the', 'one', 'true', 'reason']
1/1 [==============================] - 0s 12ms/step
Seed Sentence:  ['and', 'never', 'want', 'to', 'leave']
1/1 [==============================] - 0s 12ms/step
